In [1]:
# s - single
# p - population

# intenzifikacija
# diverzifikacija


In [3]:
# jedinka: resenje - kodiranje: 110010101011

# roditelj1 - 110010101011
# roditelj2 - 001011011100

# dete1 - 110010101100
# dete2 - 001011011011
        
# population = [jedinka...]
# while not stop_condition():
#     if elitism:
#         u novu populaciju iskopiraj nekoliko najboljih jedinki
#     izaberi jedinke za reprodukciju
#     ukrstamo
#     mutacija
#     dobili smo novu populaciju

In [1]:
# UFLP

In [2]:
import random

In [63]:
class Individual:
    def __init__(self, cost, fixedCost):
        numResources = len(fixedCost)
        self.code = [random.random() < 0.25 for _ in range(numResources)]
        self.correctNonFeasible()
        
        self.fitness = self.calcFitness(cost, fixedCost)
        
    def __lt__(self, other):
        return self.fitness < other.fitness
    
    def isFeasible(self):
        for c in self.code:
            if c:
                return True
        return False
    
    def invert(self):
        # 100 -> 000 -> 010
        randomResource = random.randrange(len(self.code))
        self.code[randomResource] = not self.code[randomResource]
        if self.isFeasible():
            return randomResource
        else:
            return -1
    
    def correctNonFeasible(self):
        for c in self.code:
            if c:
                return
        randomResource = random.randrange(len(self.code))
        self.code[randomResource] = True
        
    def calcFitness(self, cost, fixedCost):
        numUsers = len(cost)
        numResources = len(fixedCost)
        usedResources = [False for _ in range(numResources)]
        totalCost = 0
        for i in range(numUsers):
            minCost = float('inf')
            resourceUsed = -1
            for j in range(numResources):
                if cost[i][j] < minCost and self.code[j]:
                    minCost = cost[i][j]
                    resourceUsed = j
            totalCost += minCost
            usedResources[resourceUsed] = True
        # True, False, True -> True, False, False
        
        self.code = usedResources
        
        for j, resource in enumerate(self.code):
            if resource:
                totalCost += fixedCost[j]
        
        fitness = 1 / totalCost
        return fitness

In [64]:
def readInput(filename):
    with open(filename, 'r') as f:
        numUsers, numResources = [int(x) for x in f.readline().split()]
        cost = [[int(x) for x in f.readline().split()] for i in range(numUsers)]
        fixedCost = [int(x) for x in f.readline().split()]
        return cost, fixedCost

In [65]:
cost, fixedCost = readInput('../03/uflp1.txt')
cost, fixedCost

([[1, 12, 3], [2, 7, 41], [19, 21, 7]], [12, 11, 13])

In [66]:
def selection(population):
    TOURNAMENT_SIZE = 5
#     bestIndividual = max(random.sample(population, TOURNAMENT_SIZE))
    bestFitness = float('-inf')
    index = -1
    for i in range(TOURNAMENT_SIZE):
        randomIndividual = random.randrange(len(population))
        if population[randomIndividual].fitness > bestFitness:
            bestFitness = population[randomIndividual].fitness
            index = randomIndividual
    return index 

In [67]:
def crossover(parent1, parent2, child1, child2):
    breakpoint = random.randrange(len(parent1.code))
    
    child1.code[:breakpoint] = parent1.code[:breakpoint]
    child2.code[:breakpoint] = parent2.code[:breakpoint]
    
    child1.code[breakpoint:] = parent2.code[breakpoint:]
    child2.code[breakpoint:] = parent1.code[breakpoint:]
    
    child1.correctNonFeasible()
    child2.correctNonFeasible()

In [68]:
def mutation(individual):
    MUTATION_PROB = 0.05
    for i in range(len(individual.code)):
        if random.random() < MUTATION_PROB:
            individual.code[i] = not individual.code[i]
    individual.correctNonFeasible()

In [69]:
POPULATION_SIZE = 2
NUM_GENERATIONS = 1
ELITISIM_SIZE = POPULATION_SIZE // 5

population = [Individual(cost, fixedCost) for _ in range(POPULATION_SIZE)]
newPopulation = [Individual(cost, fixedCost) for _ in range(POPULATION_SIZE)]

for i in range(NUM_GENERATIONS):
    population.sort(reverse=True) # <
    for p in population:
        print(p.code, p.fitness)
    newPopulation[:ELITISIM_SIZE] = population[:ELITISIM_SIZE]
    for j in range(ELITISIM_SIZE, POPULATION_SIZE, 2):
        parent1Index = selection(population)
        parent2Index = selection(population)
        
        crossover(population[parent1Index], population[parent2Index], newPopulation[j], newPopulation[j+1])
        
        mutation(newPopulation[j])
        mutation(newPopulation[j+1])
        
        newPopulation[j].fitness = newPopulation[j].calcFitness(cost, fixedCost)
        newPopulation[j+1].fitness = newPopulation[j+1].calcFitness(cost, fixedCost)
        
    population = newPopulation
for p in population:
    print(p.code)


bestIndividual = max(population)
print(f'solution: {bestIndividual.code}, fitness: {bestIndividual.fitness}, cost: {1 / bestIndividual.fitness}')

[True, False, False] 0.029411764705882353
[True, False, True] 0.02857142857142857
[True, False, False]
[True, False, False]
solution: [True, False, False], fitness: 0.029411764705882353, cost: 34.0


In [70]:
def simulatedAnnealing(individual, cost, fixedCost, iters):
    for i in range(1, iters+1):
        # promeni malo individual
        resourceInverted = individual.invert()
        if resourceInverted < 0:
            continue
        newFitness = individual.calcFitness(cost, fixedCost)
        if newFitness > individual.fitness:
            individual.fitness = newFitness
        else:
            p = 1.0 / i ** 0.5
            q = random.uniform(0, 1)
            if p > q:
                individual.fitness = newFitness
            else:                
                # vrati se na prethodno i nastavi odatle
                individual.code[resourceInverted] = not individual.code[resourceInverted]

In [72]:
POPULATION_SIZE = 20
NUM_GENERATIONS = 10
ELITISIM_SIZE = POPULATION_SIZE // 5

population = [Individual(cost, fixedCost) for _ in range(POPULATION_SIZE)]
newPopulation = [Individual(cost, fixedCost) for _ in range(POPULATION_SIZE)]

for i in range(NUM_GENERATIONS):
    population.sort(reverse=True) # <
#     for p in population:
#         print(p.code, p.fitness)
    newPopulation[:ELITISIM_SIZE] = population[:ELITISIM_SIZE]
    for j in range(ELITISIM_SIZE, POPULATION_SIZE, 2):
        parent1Index = selection(population)
        parent2Index = selection(population)
        
        crossover(population[parent1Index], population[parent2Index], newPopulation[j], newPopulation[j+1])
        
        mutation(newPopulation[j])
        mutation(newPopulation[j+1])
        
        newPopulation[j].fitness = newPopulation[j].calcFitness(cost, fixedCost)
        newPopulation[j+1].fitness = newPopulation[j+1].calcFitness(cost, fixedCost)
    
    simulatedAnnealing(max(newPopulation), cost, fixedCost, iters=10)
    population = newPopulation
# for p in population:
#     print(p.code)


bestIndividual = max(population)
print(f'solution: {bestIndividual.code}, fitness: {bestIndividual.fitness}, cost: {1 / bestIndividual.fitness}')

solution: [True, False, False], fitness: 0.029411764705882353, cost: 34.0
